### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Uses Natural Language processing combined with REGEX to extract information from text
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/ If you run out of space in your linux directory the script will crash. There is a cache file written by selenium that may then need to be deleted to get the chrome driver working again. Follow the path in the error message and delete that cache directory

In [ ]:
pip install "numpy<2.0" 

In [ ]:
pip install spacy
#pip install -U pip setuptools wheel
#pip install -U spacy
#pip install --upgrade spacy

### Install the model at the command lines
small model
python -m spacy download en_core_web_sm

medium model
python -m spacy download en_core_web_md

large model
python -m spacy download en_core_web_lg

### create training set https://spacy.io/usage/training

This annotator works better https://agateteam.org/spacynerannotate/ combined with the DocBin code to create the .spacy file. 

https://spacy.io/usage/training#quickstart

python -m spacy init fill-config ./base_config.cfg ./config.cfg

This NER annotator worked better than the one above. Used the DocBin technique to convert to .spacy file. Spacy training command with config.cfg then worked.

Config.cfg needs to be created for the spacy command line training. 

Follow the instructions in the quickstart for the base_config.cfg file. Selected OS and clicked NER

https://spacy.io/usage/training#quickstart

Run the following at command line to create the config.cfg. 

python -m spacy init fill-config ./base_config.cfg ./config.cfg

Once the config file was done and spacy file was created run the following at command line to train the model

To debug the data:
python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./train.spacy

To debug the config file file:
python -m spacy train config.cfg --output ./output

or

python -m spacy train config.cfg 

train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

creates two directories model-best and model-last. I used model-best 

nlp1 = spacy.load(r"./output/model-best")


Notes
python -m spacy init fill-config ./base_config.cfg ./config.cfg

python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

https://spacy.io/usage/training#basics

### Train spacy if not already done. Using output https://arunmozhi.in/ner-annotator/


In [ ]:
from spacy.tokens import DocBin
import pandas as pd
import json
import os

### train the model at command line
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy 

### Main webscraper program start


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import IPython
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime
from nameparser import HumanName
import spacy
#nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")
from spacy.matcher import PhraseMatcher
from spacy import displacy

In [ ]:
#test
text2 = ("The type strain is 2205BS29-5T (=LMG 33062T =KACC 23240T), which is isolated from a marine sponge, P. elegans, "
"collected from Beomseom in Jeju-Island, Republic of Korea."

"The DNA G+C content of strain 2205BS29-5T is 67.8%. The GenBank accession numbers for the 16S rRNA gene and" 
"whole-genome sequences of strain 2205BS29-5T are OQ569368 and JAVAMQ000000000, respectively.")

In [ ]:
#test but need this later
import IPython
nlp1 = spacy.load(r"./output/model-best") #load the best model
doc = nlp1(text2)
from IPython.display import display
from spacy import displacy
from IPython.core.display import HTML, display
displacy.render(doc, style="ent")

### Find URLS from saved email in html - save as from outlook in htm format. URLs are extracted and saved as input for selenium

In [2]:
input = (r'IJSEMemail39.htm')
output = (r'NameCheckweek39.xlsx')
alldescriptions = (r'all_descriptions39')

In [3]:
#Only need this if you suspect a problem with beautiful soup not imporing the URL list correctly. Otherwise skip this step
from bs4.diagnose import diagnose
with open (input, encoding = 'unicode_escape') as f:
    data = f.read()
diagnose(data)

Diagnostic running on Beautiful Soup 4.13.3
Python version 3.11.11 (main, Jan  6 2025, 09:15:44) [GCC 8.5.0 20210514 (Red Hat 8.5.0-22)]
I noticed that html5lib is not installed. Installing it may help.
Found lxml version 5.3.1.0
Trying to parse your markup with html.parser
Here's what html.parser did with the markup:
<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:ns0="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Word.Document" name="ProgId"/>
  <meta content="Microsoft Word 15" name="Generator"/>
  <meta content="Microsoft Word 15" name="Originator"/>
  <link href="IJSEMemail39_files/filelist.xml" rel="File-List"/>
  <link href="IJSEMemail39_files/editdata.mso" rel="Edit-Time-Data"/>
  <!--[if !mso]>
<styl

In [4]:
#with open ('IJSEMemail33.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    #soup = BeautifulSoup(content, 'html.parser', from_encoding="None")
    soup = BeautifulSoup(content, 'html.parser')
#print(soup.prettify(formatter=None))
#print(soup)



In [5]:
urls = []
numberoflinks = 0
for url in soup.find_all( 'a', attrs={'href': re.compile("^https:")}):
#for url in content.find_all( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e and "join-the-society" not in e]

if numberoflinks == 0:
    raise ValueError('Number of URLs extracted is 0!')
else:
    print("Number of links found is: ", numberoflinks)
assert len(urls) > 0, "URL list cannot be empty"    
#print(urls)

Number of links found is:  12


In [6]:
def find_strains(description):
    """Find strains with spacy"""
    #print('Entering strain subroutine')
    doc = nlp(description)
    nlp1 = spacy.load(r"./output/model-best") #load the best model
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrases = ['strain', 'Strain', 'strains', 'Strains']
    patterns = [nlp(description) for description in phrases]
    phrase_matcher.add('STRAIN', None, *patterns)
               
    for sent in doc.sents:    
        for match_id, start, end in phrase_matcher(nlp(sent.text)):
            if nlp.vocab.strings[match_id] in ["STRAIN"]:
                print(sent.text)
                doc = nlp1(sent.text)
                displacy.render(doc, style="ent")
                for entity in doc.ents:
                    if entity.text not in strains:
                        entity.text.encode('ascii', 'ignore').decode('unicode_escape')
                        strains.append(entity.text)
    return strains

def remove_non_ascii(text):
    """Remove non-ASCII characters"""
    return ''.join(char for char in text if ord(char) < 128)

### Main body - Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [7]:
from selenium.webdriver.common.by import By
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
combined_description = []
for url in urls:
    counter = 1
    strains = []
    accessions = []
    orgname = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    description1 = []
    description2 = []
    snumber = []
    #scrape_with_selenium(url, counter)
    #scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        #print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        #print(author)
        author_count = len(author)
        #print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            #print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            #print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            #print(authority)
    
    #extract data from each species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description:
            strains = []
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                cleaned_text = remove_non_ascii(description)
                combined_description.append(cleaned_text)
                #print(cleaned_text)
                #print(description)
                        
                #find the organism names 
                orgname = []
                match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                if description is not None:
                    orgname = [m.group() for m in regex.finditer(description)]
                    print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    #print('accessions', accessions)
    
                #find the strains
                if description is not None:
                    find_strains(cleaned_text)
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK1')

   
    for element in driver.find_elements(By.CLASS_NAME, "tl-lowest-section"): #finds section headers
        description1 = element.text
        outer_html = element.get_attribute("outerHTML")
        if "Description of" in description1:          
            #print(outer_html)
            spans = soup.find_all('span', attrs = {'class' : 'tl-lowest-section'})
            for span in spans:
                if "Description of" in span.text:   
                    #print (span.text)
                    outer_div_id = span.find_parent('div').get('id')
                    print(f"Outer div ID: {outer_div_id}, Text: {span.text}")
                    for element in driver.find_elements(By.ID, outer_div_id):
                        description = element.text
                        cleaned_text = remove_non_ascii(description)
                        combined_description.append(cleaned_text)
                        #print(cleaned_text)
                        #print(description)
                        
                    #find the organism names   
                    orgname = []
                    match = [r'(\S+\s+){2}(?=sp. nov)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)', r'(\S+\s+){4}(?=subsp. nov.)']
                    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                    if description is not None:
                        orgname = [m.group() for m in regex.finditer(description)]
                        #print('orgname', orgname)

                    #find the accessions
                    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                    if description is not None:
                        accessions = [m.group() for m in regex.finditer(description)]
                        #print('accessions', accessions)
    
                    #find the strains
                    strains = []
                    if description is not None:
                        find_strains(cleaned_text)
                        #print('strain names', strains)
    
                    #load data into pandas dataframe
                    row_data = [orgname, accessions, strains, authority, doi, url]
                    length = len(pub_df)
                    pub_df.loc[length] = row_data
                    print('BREAK2')



        
#Close the browser window
    driver.quit()    

Brevibacterium koreense sp. nov., a moderately halophilic bacterium isolated from jogae-jeotgal, a Korean fermented seafood
orgname ['Brevibacterium koreense ']
The diagnostic diamino acid in the cell wall peptidoglycan of strain CBA3109T is meso-diaminopimelic acid.


The genomic DNA G+C content of strain CBA3109T is 62.7%.



The type strain, CBA3109T (= KACC 23387T = DSM 117564T), was isolated from jogae-jeotgal (fermented clam), a traditional fermented seafood in the Republic of Korea.


The GenBank/EMBL/DDBJ accession numbers for the 16S rRNA gene and genome sequences of strain CBA3109T are PP871682 and CP158281, respectively.
 


strain names ['CBA3109T', 'KACC 23387T', 'DSM 117564T']
orgname ['Brevibacterium koreense ']
strain names ['CBA3109T', 'KACC 23387T', 'DSM 117564T']
BREAK1
Psychrobacter saeujeotis sp. nov., a novel halophilic bacterium isolated from salted shrimp jeotgal
orgname ['Psychrobacter saeujeotis ']
This strain exhibits positive oxidase activity, hydrolyses urea and reduces nitrate to nitrite.


...

The type strain, 2305UL8-3T (=KCTC 8475T=MCCC 1K09523T=JCM 37202T), was isolated from R. bronni.


strain names ['2305UL8-3T', 'KCTC 8475T', 'MCCC 1K09523T', 'JCM 37202T']
orgname ['Roseovarius conchicola ']
strain names ['2305UL8-3T', 'KCTC 8475T', 'MCCC 1K09523T', 'JCM 37202T']
BREAK1
orgname ['Aliiroseovarius conchicola ']
The DNA G+C content of the type strain was 58.8 %.



/home/mcveigh/jupweb/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


The type strain, 2305UL8-7T (=KCTC 8476T=MCCC 1K09524T=JCM 37203T), was isolated from R. bronni.


strain names ['2305UL8-7T', 'KCTC 8476T', 'MCCC 1K09524T', 'JCM 37203T']
orgname ['Aliiroseovarius conchicola ']
strain names ['2305UL8-7T', 'KCTC 8476T', 'MCCC 1K09524T', 'JCM 37203T']
BREAK1
Request for conserving the generic name Methylacidiphilum Awala et al. 2023 over Methylacidiphilum Ratnadevi et al. 2023 to resolve the illegitimacy of Methylacidiphilum Awala et al. 2023, Methylacidiphilum caldifontis Awala et al. 2023, Methylacidiphilaceae Awala et al. 2023 and Methylacidiphilales Awala et al. 2023. Request for an Opinion


In [ ]:
#optional write description to a file
#print(combined_description)
file = open(alldescriptions, "w")
file.writelines(combined_description)
file.close()

In [8]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [', '.join(map(str, l)) for l in pub_df['Strains']]
#pub_df['Strains'] = pub_df['Strains'].astype(str) 
pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
#try drop duplicate accessions here
pub_df.explode(['PublishedName']).reset_index(drop=True)

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,Brevibacterium koreense,"[PP871682, CP158281]","CBA3109T, KACC 23387T, DSM 117564T",Nam et al. 2025,https://doi.org/10.1099/ijsem.0.006722,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006722%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681873007%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=R4vM6pg1N2kz%2BHRw4lwJMDEQiJJYZSZZkKaTBQhLd3U%3D&reserved=0
1,Psychrobacter saeujeotis,[PQ498527],"FBL11T, KACC 23745T, KCTC 8655T, JCM 37231T",Kim et al. 2025,https://doi.org/10.1099/ijsem.0.006734,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006734%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681896873%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=cbTdEp7sL4hdioYj56Thf5hg4dihRYDIG16k6yh4YJ0%3D&reserved=0
2,Marivivens marinum,"[OR581161, JAYKKJ000000000]","MK-3T, C18:17c 11-methyl, C12:1 3OH, KEMB 21417T, KCTC 8294T, JCM 36630T",Kang et al. 2025,https://doi.org/10.1099/ijsem.0.006735,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006735%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681923750%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=4R571NJ5zn%2FEb1oAY50qzFjeAyWaWArpWpZjJPUbShk%3D&reserved=0
3,Roseovarius conchicola,"[PP346233, JBGEHK000000000]","2305UL8-3T, KCTC 8475T, MCCC 1K09523T, JCM 37202T",Ha et al. 2025,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0
4,Aliiroseovarius conchicola,"[PP346247, JBGEHL000000000]","2305UL8-7T, KCTC 8476T, MCCC 1K09524T, JCM 37203T",Ha et al. 2025,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0


In [ ]:
#pub_df['Strains'] = pub_df['Strains'].astype(pd.StringDtype())
#pub_df['Strains'] = pub_df['Strains'].str.replace(',', ', ')
#pub_df

In [9]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
#pub2_df
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
#pub4_df

In [10]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
#df_unique = df_unique.dropna()
#df_unique

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [11]:
#df_unique['accession'] = df_unique['accession'].astype('str') 
df_unique.loc[:, 'accession'] = df_unique['accession'].astype('str') 

In [12]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [13]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


0

In [14]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna(subset=['NCBIname'])
srcchk_df 

,accession,NCBIname,taxid,strain
0,PP871682,Brevibacterium sp. CBA3109,3140787,CBA3109
1,CP158281,Brevibacterium sp. CBA3109,3140787,CBA3109
2,PQ498527,Psychrobacter sp.,56811,FBL11
3,OR581161,Salipiger sp.,2078585,MK-3
4,JAYKKJ000000000,Roseobacteraceae bacterium MK3,3110173,MK3
5,PP346233,Roseovarius sp.,1486281,2305UL8-3
6,JBGEHK000000000,Roseovarius sp. 2305UL8-3,3121636,2305UL8-3
7,PP346247,Aliiroseovarius sp.,1872442,2305UL8-7
8,JBGEHL000000000,Aliiroseovarius sp. 2305UL8-7,3121637,2305UL8-7


### Combine dataframes into one

In [15]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Brevibacterium koreense,Brevibacterium sp. CBA3109,"CBA3109T, KACC 23387T, DSM 117564T",CP158281,CBA3109,Nam et al. 2025,3140787,https://doi.org/10.1099/ijsem.0.006722,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006722%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681873007%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=R4vM6pg1N2kz%2BHRw4lwJMDEQiJJYZSZZkKaTBQhLd3U%3D&reserved=0
1,Marivivens marinum,Roseobacteraceae bacterium MK3,"MK-3T, C18:17c 11-methyl, C12:1 3OH, KEMB 21417T, KCTC 8294T, JCM 36630T",JAYKKJ000000000,MK3,Kang et al. 2025,3110173,https://doi.org/10.1099/ijsem.0.006735,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006735%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681923750%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=4R571NJ5zn%2FEb1oAY50qzFjeAyWaWArpWpZjJPUbShk%3D&reserved=0
2,Roseovarius conchicola,Roseovarius sp. 2305UL8-3,"2305UL8-3T, KCTC 8475T, MCCC 1K09523T, JCM 37202T",JBGEHK000000000,2305UL8-3,Ha et al. 2025,3121636,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0
3,Aliiroseovarius conchicola,Aliiroseovarius sp. 2305UL8-7,"2305UL8-7T, KCTC 8476T, MCCC 1K09524T, JCM 37203T",JBGEHL000000000,2305UL8-7,Ha et al. 2025,3121637,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0
4,Marivivens marinum,Salipiger sp.,"MK-3T, C18:17c 11-methyl, C12:1 3OH, KEMB 21417T, KCTC 8294T, JCM 36630T",OR581161,MK-3,Kang et al. 2025,2078585,https://doi.org/10.1099/ijsem.0.006735,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006735%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681923750%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=4R571NJ5zn%2FEb1oAY50qzFjeAyWaWArpWpZjJPUbShk%3D&reserved=0
5,Roseovarius conchicola,Roseovarius sp.,"2305UL8-3T, KCTC 8475T, MCCC 1K09523T, JCM 37202T",PP346233,2305UL8-3,Ha et al. 2025,1486281,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de99

In [16]:
def highlight_rows(row):
    ijsemvalue = row.loc['PublishedName']
    ncbivalue = row.loc['NCBIname']
    if ijsemvalue != ncbivalue:
        color = '#FFB3BA' # Red
    elif ijsemvalue == ncbivalue:
        color = '#BAFFC9' # Green
    return ['background-color: {}'.format(color) for r in row]

new_df = combine_df.style.apply(highlight_rows, axis=1, subset=['PublishedName', 'NCBIname'])
new_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Brevibacterium koreense,Brevibacterium sp. CBA3109,"CBA3109T, KACC 23387T, DSM 117564T",CP158281,CBA3109,Nam et al. 2025,3140787,https://doi.org/10.1099/ijsem.0.006722,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006722%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681873007%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=R4vM6pg1N2kz%2BHRw4lwJMDEQiJJYZSZZkKaTBQhLd3U%3D&reserved=0
1,Marivivens marinum,Roseobacteraceae bacterium MK3,"MK-3T, C18:17c 11-methyl, C12:1 3OH, KEMB 21417T, KCTC 8294T, JCM 36630T",JAYKKJ000000000,MK3,Kang et al. 2025,3110173,https://doi.org/10.1099/ijsem.0.006735,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006735%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681923750%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=4R571NJ5zn%2FEb1oAY50qzFjeAyWaWArpWpZjJPUbShk%3D&reserved=0
2,Roseovarius conchicola,Roseovarius sp. 2305UL8-3,"2305UL8-3T, KCTC 8475T, MCCC 1K09523T, JCM 37202T",JBGEHK000000000,2305UL8-3,Ha et al. 2025,3121636,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0
3,Aliiroseovarius conchicola,Aliiroseovarius sp. 2305UL8-7,"2305UL8-7T, KCTC 8476T, MCCC 1K09524T, JCM 37203T",JBGEHL000000000,2305UL8-7,Ha et al. 2025,3121637,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681947001%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=k4eYcUD0E2D7naxsxy4yID4nnDWC6Qi8XR7toQm919M%3D&reserved=0
4,Marivivens marinum,Salipiger sp.,"MK-3T, C18:17c 11-methyl, C12:1 3OH, KEMB 21417T, KCTC 8294T, JCM 36630T",OR581161,MK-3,Kang et al. 2025,2078585,https://doi.org/10.1099/ijsem.0.006735,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006735%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de9900a%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638787572681923750%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C40000%7C%7C%7C&sdata=4R571NJ5zn%2FEb1oAY50qzFjeAyWaWArpWpZjJPUbShk%3D&reserved=0
5,Roseovarius conchicola,Roseovarius sp.,"2305UL8-3T, KCTC 8475T, MCCC 1K09523T, JCM 37202T",PP346233,2305UL8-3,Ha et al. 2025,1486281,https://doi.org/10.1099/ijsem.0.006725,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006725%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C71114c9c0f334e38e12708dd6de99

### write output to excel

In [ ]:
new_df.to_excel(output, engine='xlsxwriter', index = False, na_rep = '') 

In [ ]:
combine_df.dtypes